In [3]:
print('start')
print('征信模板pdf提取函数如下：')

start
征信模板pdf提取函数如下：


In [5]:
# 读取PDF
import PyPDF2
# 分析PDF的layout，提取文本
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# 从PDF的表格中提取文本和表格
import pdfplumber
import sys

#创建一个文本提取函数
def text_extraction(element):
    # 从行元素中提取文本
    line_text = element.get_text().strip('\n')
    line_text = line_text.replace("\n",": ")
    # 探析文本的格式
    #用文本行中出现的所有格式初始化列表
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            #遍历文本行中的每个字符
            for character in text_line:##
                if isinstance(character, LTChar):
                    #追加字符的font-family
                    line_formats.append(character.fontname)
                    # 追加字符的font-size
                    line_formats.append(character.size)
        #找到行中唯一的字体大小和名称
        format_per_line = list(set(line_formats))
        # 返回包含每行文本及其格式的元组
    return (line_text+'\n', format_per_line)

In [6]:
#从页面中提取表格内容
def extract_table(pdf_path, page_num, table_num):
    # 打开PDF文件
    pdf = pdfplumber.open(pdf_path)
    # 查找已检查的页面
    table_page = pdf.pages[page_num]
    # 提取适当的表格
    table = table_page.extract_tables()[table_num]
    return table

# 将表格转换为适当的格式
def table_converter(table):
    table_string = ''
    # 遍历表格的每一行
    for row_num in range(len(table)):
        row = table[row_num]
        # 从warp的文字删除线路断路器
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # 将表格转换为字符串，注意'|'、'\n'
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')#
        # 删除最后一个换行符
        table_string = table_string[:-1]
    return table_string


In [7]:
def PDF_extract(): ##############################################
    # 查找PDF路径
    pdf_path = 'F:/浙商nlp项目/征信模板.pdf'
    # 创建字典以从每个图像中提取文本
    text_per_page = {}
    # 打开pdf文件
    pdf = pdfplumber.open(pdf_path)
    # 我们从PDF中提取页面
    pages_info_all = list(extract_pages(pdf_path))
    for pagenum, page in enumerate(pages_info_all):  # [:1]

        # 初始化从页面中提取文本所需的变量
        page_text = []
        line_format = []
        text_from_tables = []
        page_content = []

        # 初始化检查表的数量
        table_num = 0
        first_element = True
        table_extraction_flag = False

        # 查找已检查的页面
        page_tables = pdf.pages[pagenum]
        # 找出本页上的表格数目
        tables = page_tables.find_tables()
        # 找出本页表格边界
        upper_side_list = [page.bbox[3] - tables[table_num].bbox[1] for table_num in range(len(tables))]
        lower_side_list = [page.bbox[3] - tables[table_num].bbox[3] for table_num in range(len(tables))]

        # 找到所有的元素
        page_elements = [(element.y1, element) for element in page._objs]

        # 对页面中出现的所有元素进行排序，感觉不需要吧
        page_elements.sort(key=lambda a: a[0], reverse=True)

        # 查找组成页面的元素
        for i, component in enumerate(page_elements):
            # 提取PDF中元素顶部的位置
            # pos= component[0]
            # 提取页面布局的元素
            element = component[1]

            # 检查表的元素
            if isinstance(element, LTRect):
                table_content = []

                # 如果第一个矩形元素
                if first_element == True and (table_num + 1) <= len(tables) and element.y1 <= upper_side_list[table_num]:
                    # 从表中提取信息
                    table = extract_table(pdf_path, pagenum, table_num)
                    # 将表信息转换为结构化字符串格式
                    table_string = table_converter(table)
                    # 将表字符串追加到列表中
                    table_content.append(table)
                    table_content.append(table_string)
                    text_from_tables.append(table_content)
                    page_content.append(table)
                    # 将标志设置为True以再次避免该内容
                    table_extraction_flag = True
                    # 让它成为另一个元素
                    table_num += 1

                # 检查我们是否是上个表内信息
                if element.y0 >= lower_side_list[table_num - 1] and element.y1 <= upper_side_list[table_num - 1]:
                    pass
                elif not isinstance(page_elements[i + 1][1], LTRect):
                    table_extraction_flag = False
                    first_element = True

            # 检查该元素是否为文本元素
            if isinstance(element, LTTextContainer):
                # 检查文本是否出现在表中
                if table_extraction_flag == False:
                    # 使用该函数提取每个文本元素的文本和格式
                    (line_text, format_per_line) = text_extraction(element)
                    # 将每行的文本追加到页文本
                    page_text.append(line_text)
                    # 附加每一行包含文本的格式
                    line_format.append(format_per_line)
                    page_content.append(line_text)
                else:
                    # 省略表中出现的文本
                    pass

        # 创建字典的键
        dctkey = 'Page_' + str(pagenum)
        # 将list的列表添加为页键的值
        text_per_page[dctkey] = [page_text, line_format, text_from_tables, page_content]

    return text_from_tables

In [8]:
extract=PDF_extract()

In [11]:
print(extract[1])

[[['后付费业务的还款状态说明：'], ['电信业务：\n*-服务已开通但本月不需缴费；\nN-正常（用户已经按时缴纳该月全部服务费用，包括在宽限期内的用户）；\n0-欠费超过宽限期不足1个月；\n1-欠费超过宽限期1个月不足2个月；\n2-欠费超过宽限期2个月不足3个月；\n3-欠费超过宽限期3个月不足4个月；\n4-欠费超过宽限期4个月不足5个月；\n5-欠费超过宽限期5个月不足6个月；\n6-欠费超过宽限期6个月以上；\nC-正常销户（结清后的销户）； G-结束（非正常结清的销户）；\n#-未知：没有此期数据。']], '|后付费业务的还款状态说明：||电信业务： *-服务已开通但本月不需缴费； N-正常（用户已经按时缴纳该月全部服务费用，包括在宽限期内的用户）； 0-欠费超过宽限期不足1个月； 1-欠费超过宽限期1个月不足2个月； 2-欠费超过宽限期2个月不足3个月； 3-欠费超过宽限期3个月不足4个月； 4-欠费超过宽限期4个月不足5个月； 5-欠费超过宽限期5个月不足6个月； 6-欠费超过宽限期6个月以上； C-正常销户（结清后的销户）； G-结束（非正常结清的销户）； #-未知：没有此期数据。|']


In [12]:
print('为了将其中的表格进行json表示，将得到的表格数据进行处理，先用示例')

为了将其中的表格进行json表示，将得到的表格数据进行处理，先用示例


In [16]:
print('由于表格中含有较多混合表格，涉及同一表格内的上下对齐与左右对齐，目前没有想到能够准确判断什么时候纵向匹配，什么时候横向匹配，')
print('就先拿表格中出现较多的“编号”为标志进行筛选,形成表格的json格式代码如下')

由于表格中含有较多混合表格，涉及同一表格内的上下对齐与左右对齐，目前没有想到能够准确判断什么时候纵向匹配，什么时候横向匹配，
就先拿表格中出现较多的“编号”为标志进行筛选,形成表格的json格式代码如下


In [31]:
def merge_dict(d):#检查，进行列横行对应
    #d = {'证件类型': '护照', '证件号码': 'G30003516', '被查询者证件号码': '110108******181X', '查询机构': '中国人民银行营业管理部', '护照': '军官证', 'G30003516': 'M0938981'}  # {'证件类型': '护照', '护照': '军官证'}
    result = {}
    seen_values = set()

    for key, value in d.items():
        if isinstance(value,list):
            result[key]=value
            return result
        elif isinstance(value,str):
            if value in seen_values:
                continue
            seen_values.add(value)
            if value in d:
                result[key] = [value, d[value]]
            elif key in seen_values:
                continue
            else:
                result[key] = [value]
    return result
# merged_dict = merge_dict()

In [18]:
def table2json1(lis):#不能表格行数少于三行，否则直接跳出循环
    dicc = {}
    for linea in range(int(len(lis)/2-1)):#列竖行对应
        for t,z in enumerate(lis[2*linea]):
            if z == None:#表格合并，无值情况
                continue
            elif '：' in z:#有冒号情况
                str, st = z.split('：')
                dicc[str] = st
            elif len(z) == 0:#‘’情况
                continue
            elif '编号' in z:#横行读情况
                ss = lis[linea*2:]  # 有个问题，要是横行对应的不到表格末端也会有问题
                #transposed = [[row[i] for row in ss] for i in range(len(ss[0]))]#列表转秩
                zilis = []
                tem = {}
                dicc['biao_title'] = zilis
                for r in range(int(len(ss)-1)):#列
                    for a, b in enumerate(ss[r+1]):#行  第一行做表头
                        if b == None:#表格合并，无值情况
                            continue
                        elif len(b) == 0:#‘’情况
                            continue
                        else:
                            tem[ss[0][a]]=b
                    temp = tem.copy()
                    dicc['biao_title'].append(temp)
                    tem={}
                return dicc
            else:
                dicc[z] = lis[2*linea + 1][t]
    return dicc

In [3]:
print('测试单条列表，以PDF中不同形式的表格分别为例子，进行测试')

测试单条列表，以PDF中不同形式的表格分别为例子，进行测试


In [13]:
s = [['编号', '居住地址', '住宅电话', '居住状况', '信息更新日期'],
     ['1', '北京市朝阳区春晓园北区7号楼C555室', '010—83234323', '按揭', '2015.05.01'],
     ['2', '北京市海淀区学院路20号院甲2号楼401室', '010—62123212', '集体宿舍', '2015.01.22'],
     ['3', '北京市西城区复兴门北大街4号楼603室', '010—59873232', '租房', '2014.02.01'], 
     ['4', '北京市西城区西交民巷64号院1号楼201室', '010—59000011', '租房', '2014.01.22'], 
     ['5', '北京市海淀区花园路10号院3号楼701室', '010—57889900', '租房', '2013.11.12'], 
     ['编号', '数据发生机构名称', None, None, None], 
     ['1', '工商银行', None, None, None], 
     ['2', '中国银行', None, None, None], 
     ['3', '农业银行', None, None, None], 
     ['4', '中信银行', None, None, None], 
     ['5', '建设银行', None, None, None]]

In [21]:
tt= [['编号', '工作单位', '单位性质', '单位地址', None, None, '单位电话'], 
     ['1', '中国人民银行征信中心企业部', '机关、事业单位', '北京市西城区金融大街35号\\n国际企业大厦A座305室', None, None, '010—83233333'], 
     ['2', '北京大学信息科学技术学院软件\\n工程研究所', '机关、事业单位', '北京市海淀区颐和园路5号\\n理科3号楼601室', None, None, '010—62123333'],
     ['3', '中科院微电子研究所', '机关、事业单位', '北京市海淀区清华园1号', None, None, '010—59875555'],
     ['4', '北京威力公司', '个体、私营企业', '北京市朝阳区北土城西路3号', None, None, '010—59006666'], 
     ['5', 'IBM软件中心', '外资企业', '北京市朝阳区北辰东路8号', None, None, '010—57889900'], 
     ['编号', '职业', '行业', '职务', '职称', '进入本单位年份', '信息更新日期'], 
     ['1', '办事人员和有关人员', '金融业', '中级领导', '中级', '2008', '2015.05.01'], 
     ['2', '专业技术人员', '科学研究和技术服务业', '中级领导', '中级', '2007', '2015.01.22'],
     ['3', '专业技术人员', '科学研究和技术服务业', '一般员工', '初级', '2004', '2014.02.01'], 
     ['4', '专业技术人员', '科学研究和技术服务业', '一般员工', '初级', '2003', '2014.01.22'], 
     ['5', '专业技术人员', '科学研究和技术服务业', '一般员工', '初级', '2001', '2013.11.12'], 
     ['编号', '数据发生机构名称', None, None, None, None, None],
     ['1', '工商银行', None, None, None, None, None],
     ['2', '中国银行', None, None, None, None, None], 
     ['3', '农业银行', None, None, None, None, None], 
     ['4', '中信银行', None, None, None, None, None], 
     ['5', '建设银行', None, None, None, None, None]]

In [6]:
q = [['', '防欺诈警示', None, ''],
     ['信息主体申请设置防欺诈警示，联系电话：010—90000000/13900000000。', None, None, None],
     ['生效日期', None, '截止日期', None],
     ['2015年5月29日', None, '2016年5月28日', None]]

In [8]:
ll=[['编号', '手机号码', None, '信息更新日期', '数据发生机构名称'],
      ['1', '13811117777', None, '2015.05.01', '工商银行'],
      ['2', '13811112222', None, '2015.01.22', '中国银行'],
      ['3', '13900000000', None, '2014.02.01', '农业银行'],
      ['4', '13300000000', None, '2014.01.22', '中信银行'],
      ['5', '18600000000', None, '2013.11.12', '建设银行']]

In [9]:
gg = [['性别', None, '出生日期', '婚姻状况', '就业状况'],
      ['男', None, '1981.08.15', '已婚', '在职'],
      ['数据发生机构名称', None, '数据发生机构名称', '数据发生机构名称', '数据发生机构名称'],
      ['工商银行', None, '工商银行', '工商银行', '工商银行'],
      ['学历', None, '学位', '国籍', '电子邮箱'],
      ['本科', None, '学士', '中国', 'zhang15@hotmail.com'],
      ['数据发生机构名称', None, '数据发生机构名称', '数据发生机构名称', '数据发生机构名称'],
      ['工商银行', None, '工商银行', '工商银行', '工商银行'],
      ['通讯地址', None, None, '户籍地址', None],
      ['北京市西城区金融大街35号国际企业大厦A座301室', None, None, '北京市朝阳区春晓园北区7号楼C522室', None],
      ['数据发生机构名称', None, None, '数据发生机构名称', None],
      ['工商银行', None, None, '工商银行', None],
      ['编号', '手机号码', None, '信息更新日期', '数据发生机构名称'],
      ['1', '13811117777', None, '2015.05.01', '工商银行'],
      ['2', '13811112222', None, '2015.01.22', '中国银行'],
      ['3', '13900000000', None, '2014.02.01', '农业银行'],
      ['4', '13300000000', None, '2014.01.22', '中信银行'],
      ['5', '18600000000', None, '2013.11.12', '建设银行']]

In [19]:
new = table2json1(gg)
print(new)

{'性别': '男', '出生日期': '1981.08.15', '婚姻状况': '已婚', '就业状况': '在职', '数据发生机构名称': '工商银行', '学历': '本科', '学位': '学士', '国籍': '中国', '电子邮箱': 'zhang15@hotmail.com', '通讯地址': '北京市西城区金融大街35号国际企业大厦A座301室', '户籍地址': '北京市朝阳区春晓园北区7号楼C522室', 'biao_title': [{'编号': '1', '手机号码': '13811117777', '信息更新日期': '2015.05.01', '数据发生机构名称': '工商银行'}, {'编号': '2', '手机号码': '13811112222', '信息更新日期': '2015.01.22', '数据发生机构名称': '中国银行'}, {'编号': '3', '手机号码': '13900000000', '信息更新日期': '2014.02.01', '数据发生机构名称': '农业银行'}, {'编号': '4', '手机号码': '13300000000', '信息更新日期': '2014.01.22', '数据发生机构名称': '中信银行'}, {'编号': '5', '手机号码': '18600000000', '信息更新日期': '2013.11.12', '数据发生机构名称': '建设银行'}]}


In [28]:
dic = merge_dict(new)#检查，进行列横行对应
print(dic)

{'性别': ['男'], '出生日期': ['1981.08.15'], '婚姻状况': ['已婚'], '就业状况': ['在职'], '数据发生机构名称': ['工商银行'], '学历': ['本科'], '学位': ['学士'], '国籍': ['中国'], '电子邮箱': ['zhang15@hotmail.com'], '通讯地址': ['北京市西城区金融大街35号国际企业大厦A座301室'], '户籍地址': ['北京市朝阳区春晓园北区7号楼C522室'], 'biao_title': [{'编号': '1', '手机号码': '13811117777', '信息更新日期': '2015.05.01', '数据发生机构名称': '工商银行'}, {'编号': '2', '手机号码': '13811112222', '信息更新日期': '2015.01.22', '数据发生机构名称': '中国银行'}, {'编号': '3', '手机号码': '13900000000', '信息更新日期': '2014.02.01', '数据发生机构名称': '农业银行'}, {'编号': '4', '手机号码': '13300000000', '信息更新日期': '2014.01.22', '数据发生机构名称': '中信银行'}, {'编号': '5', '手机号码': '18600000000', '信息更新日期': '2013.11.12', '数据发生机构名称': '建设银行'}]}


In [20]:
new2 = table2json1(s)
print(new2)

{'biao_title': [{'编号': '1', '居住地址': '北京市朝阳区春晓园北区7号楼C555室', '住宅电话': '010—83234323', '居住状况': '按揭', '信息更新日期': '2015.05.01'}, {'编号': '2', '居住地址': '北京市海淀区学院路20号院甲2号楼401室', '住宅电话': '010—62123212', '居住状况': '集体宿舍', '信息更新日期': '2015.01.22'}, {'编号': '3', '居住地址': '北京市西城区复兴门北大街4号楼603室', '住宅电话': '010—59873232', '居住状况': '租房', '信息更新日期': '2014.02.01'}, {'编号': '4', '居住地址': '北京市西城区西交民巷64号院1号楼201室', '住宅电话': '010—59000011', '居住状况': '租房', '信息更新日期': '2014.01.22'}, {'编号': '5', '居住地址': '北京市海淀区花园路10号院3号楼701室', '住宅电话': '010—57889900', '居住状况': '租房', '信息更新日期': '2013.11.12'}, {'编号': '编号', '居住地址': '数据发生机构名称'}, {'编号': '1', '居住地址': '工商银行'}, {'编号': '2', '居住地址': '中国银行'}, {'编号': '3', '居住地址': '农业银行'}, {'编号': '4', '居住地址': '中信银行'}, {'编号': '5', '居住地址': '建设银行'}]}


In [22]:
new = table2json1(tt)
print(new)

{'biao_title': [{'编号': '1', '工作单位': '中国人民银行征信中心企业部', '单位性质': '机关、事业单位', '单位地址': '北京市西城区金融大街35号\\n国际企业大厦A座305室', '单位电话': '010—83233333'}, {'编号': '2', '工作单位': '北京大学信息科学技术学院软件\\n工程研究所', '单位性质': '机关、事业单位', '单位地址': '北京市海淀区颐和园路5号\\n理科3号楼601室', '单位电话': '010—62123333'}, {'编号': '3', '工作单位': '中科院微电子研究所', '单位性质': '机关、事业单位', '单位地址': '北京市海淀区清华园1号', '单位电话': '010—59875555'}, {'编号': '4', '工作单位': '北京威力公司', '单位性质': '个体、私营企业', '单位地址': '北京市朝阳区北土城西路3号', '单位电话': '010—59006666'}, {'编号': '5', '工作单位': 'IBM软件中心', '单位性质': '外资企业', '单位地址': '北京市朝阳区北辰东路8号', '单位电话': '010—57889900'}, {'编号': '编号', '工作单位': '职业', '单位性质': '行业', '单位地址': '职务', None: '进入本单位年份', '单位电话': '信息更新日期'}, {'编号': '1', '工作单位': '办事人员和有关人员', '单位性质': '金融业', '单位地址': '中级领导', None: '2008', '单位电话': '2015.05.01'}, {'编号': '2', '工作单位': '专业技术人员', '单位性质': '科学研究和技术服务业', '单位地址': '中级领导', None: '2007', '单位电话': '2015.01.22'}, {'编号': '3', '工作单位': '专业技术人员', '单位性质': '科学研究和技术服务业', '单位地址': '一般员工', None: '2004', '单位电话': '2014.02.01'}, {'编号': '4', '工作单位': '专业技术人员', '单位性质': '科

In [23]:
print('对于行数较少的列表可以用以下函数处理')

对于行数较少的列表可以用以下函数处理


In [24]:
def table2json2(lis):#表格行数较少，不会出现横行对应情况
    dicc = {}
    for linea in range(int(len(lis)-1)):#列竖行对应
        for t,z in enumerate(lis[linea]):
            if z == None:#表格合并，无值情况
                continue
            elif '：' in z:#有冒号情况
                str, st = z.split('：')
                dicc[str] = st
            elif len(z) == 0:#‘’情况
                continue
            else:
                dicc[z] = lis[linea + 1][t]
    return dicc

In [25]:
vv = [['报告编号：201506010000332123', None, None, '报告时间：2015.06.01 10:05:15', None],
    ['被查询者姓名', '被查询者证件类型', '被查询者证件号码', '查询机构', '查询原因'],
    ['张十五', '身份证', '110108******181X', '中国人民银行营业管理部', '本人查询（临柜）']]

In [26]:
print(table2json2(vv))

{'报告编号': '201506010000332123', '报告时间': '2015.06.01 10:05:15', '被查询者姓名': '张十五', '被查询者证件类型': '身份证', '被查询者证件号码': '110108******181X', '查询机构': '中国人民银行营业管理部', '查询原因': '本人查询（临柜）'}


In [27]:
xv = [['', '其他证件信息', None, ''],
     ['证件类型', None, '证件号码', None],
     ['护照', None, 'G30003516', None],
     ['军官证', None, 'M0938981', None]]

In [28]:
print(table2json2(xv))

{'其他证件信息': None, '证件类型': '护照', '证件号码': 'G30003516', '护照': '军官证', 'G30003516': 'M0938981'}


In [29]:
print('行数少的情况需要进行横行检查，防止单行上下对应与表格现实情况不符合')

行数少的情况需要进行横行检查，防止单行上下对应与表格现实情况不符合


In [32]:
dic2 = merge_dict(table2json2(xv))#检查，进行列横行对应
print(dic2)

{'证件类型': ['护照', '军官证'], '证件号码': ['G30003516', 'M0938981']}
